# Extracción de características TF-IDF

Primero importamos todas las librerías necesarias

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import spacy
import gensim

pd.options.display.max_colwidth = None


Usamos el mismo conjunto de textos de ejemplo *(CORPUS)*

In [ ]:
corpus = ['El cielo es azul y bonito',
          'Me encanta el cielo azul, pero no el cielo plomizo',
          'Bonito cielo hacía ese día',
          'Hoy he desayunado huevos con jamón y tostadas',
          'Juan odia las tostadas y los huevos con jamón',
          'las tostadas de jamón están muy buenas']

## Limpieza del texto
Definimos una función simple de limpieza y normalización del texto y la aplicamos a nuestro corpus.

In [ ]:
nlp = spacy.load("es_core_news_md")
def normalizar_doc(doc):
    '''Función que normaliza un texto cogiendo sólo
    las palabras en minúsculas mayores de 3 caracteres'''
    # separamos en tokens
    tokens = nlp(doc)
    # filtramos stopwords
    filtered_tokens = [t.lower_ for t in tokens if
                       len(t.text)>3 and
                       not t.is_space and
                       not t.is_punct]
    # juntamos de nuevo en una cadena
    doc = ' '.join(filtered_tokens)
    return doc

In [ ]:
norm_corpus = list(map(normalizar_doc, corpus))
norm_corpus

# Librería `scikit-learn`

## Modelo TF-IDF
Este modelo promedia la frecuencia de aparición de cada término (TF) por el número de documentos en los que aparece el término (IDF).

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(norm=None)
tv_matrix = tv.fit_transform(norm_corpus)
tv_matrix.shape

In [ ]:
#también es una matriz sparse
tv_matrix

Tenemos los mismos atributos que en el CountVectorizer

In [ ]:
tv.get_feature_names_out()

El vocabulario que ha aprendido es el mismo que en el caso del BoW:

In [ ]:
tv.vocabulary_

Al calcular la matriz de vectores de documento se aplica un peso a cada término en función de su IDF:

In [ ]:
tv_matrix = tv_matrix.toarray()
vocab = tv.get_feature_names_out()
pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

In [ ]:
#pesos para cada término (valor idf(t))
tv.idf_

In [ ]:
[(palabra, peso) for palabra, peso in zip(tv.get_feature_names_out(), tv.idf_)]

La frecuencia de documentos para cada término (valor tf(término, documento)) no se almacena directamente en el vectorizador pero se puede calcular:

In [ ]:
df = np.sum(tv_matrix>0, axis=0)
df

In [ ]:
#Frec. de documentos y peso IDF para cada término
pd.DataFrame({'término':vocab, 'freq':df, 'idf':tv.idf_})

In [ ]:
#La matriz TF-IDF es la BoW multiplicada por los pesos IDF
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
cv_matrix = cv.fit_transform(norm_corpus).toarray()
cv_matrix

In [ ]:
pd.DataFrame(np.round(cv_matrix*tv.idf_, 2), columns=vocab)

Cálculo de los pesos IDF

In [ ]:
#fórmula de scikit-learn
#idf(t) = log [ (1 + n) / (1 + df(t)) ] + 1
n = tv_matrix.shape[0]
np.log((n+1)/(1+df))+1

In [ ]:
#fórmula estándar para TF-IDF
#idf(t) = log [ n / (df(t)] + 1 
np.log(n/(df))+1

Si normalizamos, se ajustan los valores tf-idf en cada documento según la norma 'l2' (suma de cuadrados) o 'l1' (suma de valores absolutos)

In [ ]:
tv_l2 = TfidfVectorizer(norm='l2', use_idf=True)
tv_matrix_l2 = tv_l2.fit_transform(norm_corpus).toarray()
pd.DataFrame(np.round(tv_matrix_l2, 2), columns=tv_l2.get_feature_names_out())

In [ ]:
np.sqrt(np.sum(tv_matrix_l2**2, axis=1)) #cada fila está normalizada a uno (norma 'L2')

In [ ]:
np.sqrt(np.sum(tv_matrix**2, axis=1)) #valores de cada documento sin normalizar

## Cálculo de la matriz en nuevos documentos
Hay que aplicar el método `transform` siempre que queremos vectorizar un nuevo conjunto de documentos.\
Al calcular la matriz TF-IDF para el nuevo corpus, el peso de cada término (IDF) no se modifica

In [ ]:
nuevo_corpus = ['El Cielo amenaza lluvia', 'Pedro desayuna tostadas de jamón con tomate']
norm_nuevo_corpus = list(map(normalizar_doc, nuevo_corpus))
new_matrix=tv.transform(norm_nuevo_corpus).toarray()
pd.DataFrame(np.round(new_matrix, 2), columns=vocab)

Si aplicamos una vectorización normalizada, se normaliza cada documento considerando sólo los términos del vocabulario.
### Ejercicio 1
Aplica la vectorización TF-IDF con la normalización *l2* entrenada con el corpus de ejemplo al nuevo corpus y muestra la matriz TF-IDF generada.

### Modelo n-gramas
Con el vectorizador `tfidfvectorizer` también podemos especificar el rango de n-gramas y el `min_df`.
### Ejercicio 2
Calcula la matriz TF-IDF para el corpus de ejemplo considerando unigramas y bigramas pero sólo para los términos que aparecen al menos en 2 documentos. No apliques ninguna normalización tras vectorizar.

# Librería `Gensim`
Para trabajar con la librería `Gensim` es necesario transformar los documentos en una lista de tokens. El modelo TF-IDF se calcula a partir del BoW. 

In [ ]:
def normalizar_doc_tokenize(doc):
    '''Función que normaliza un texto cogiendo sólo
    las palabras en minúsculas mayores de 3 caracteres'''
    # separamos en tokens
    tokens = nlp(doc)
    # filtramos stopwords
    filtered_tokens = [t.lower_ for t in tokens if
                       len(t.text)>3 and
                       not t.is_space and
                       not t.is_punct]

    return filtered_tokens

Convertimos nuestros texto de ejemplo en una lista de tokens y visualizamos el primer documento como ejemplo:

In [ ]:
tokenized_corpus = map(normalizar_doc_tokenize, corpus)
tokenized_corpus

Para calcular la matriz TF-IDF primero hay que calcular el modelo BoW:

Primero aprendemos las palabras y luego generamos la matriz sobre el `corpus` que queramos:

In [ ]:
from gensim.corpora import Dictionary

diccionario = Dictionary(tokenized_corpus)

In [ ]:
len(diccionario.token2id)

In [ ]:
#hay que volver a generar el objeto map
mapped_corpus = [diccionario.doc2bow(text)
                 for text in map(normalizar_doc_tokenize, corpus)]

In [ ]:
mapped_corpus

In [ ]:
for (i, tf) in mapped_corpus[1]:
    print(f"{diccionario[i]}: {tf}")

El objeto `Dictionary` guarda la frecuencia de documentos de cada término (núm. de documentos en los que aparece) en el atributo `dfs`:

In [ ]:
for i in diccionario.dfs:
    print(f"{diccionario[i]}: {diccionario.dfs[i]}")

## Modelo TF-IDF
Hay que hacer una transformación sobre la matriz BoW

In [ ]:
from gensim.models import TfidfModel

tfidf = TfidfModel(mapped_corpus)
corpus_tfidf = tfidf[mapped_corpus]

De nuevo, la librería `gensim` genera por cada documento una lista de tuplas (ID,frecuencia) donde ahora la frecuencia está normalizada por la inversa de la frecuencia de documentos que contienen el término:

In [ ]:
corpus_tfidf

El modelo devuelve un objeto `TransformedCorpus` que se puede recorrer como un *iterable* o indexar directamente:

In [ ]:
len(corpus_tfidf)

In [ ]:
corpus_tfidf[1]

In [ ]:
for (i, v) in corpus_tfidf[1]:
    print(f"{diccionario[i]}: {v:.2f}")

## Aplicación de los modelos a nuevos textos
Para aplicar un modelo BoW o TF-IDF a un nuevo documento hay que utilizar los modelos ya entrenados en `gensim` sobre el corpus original. Hay que calcular el BoW del nuevo corpus con el objeto `Dictionary` original y sobre esta matriz calcular su TF-IDF con el modelo `TfidfModel` entrenado con el corpus original:

In [ ]:
mapped_nuevo_corpus = [diccionario.doc2bow(text)
                 for text in map(normalizar_doc_tokenize, nuevo_corpus)]
#BoW
mapped_nuevo_corpus

In [ ]:
#TF-IDF
nuevo_corpus_tfidf = tfidf[mapped_nuevo_corpus]
nuevo_corpus_tfidf

In [ ]:
len(nuevo_corpus_tfidf)

In [ ]:
list(nuevo_corpus_tfidf)

In [ ]:
for (i, v) in nuevo_corpus_tfidf[1]:
    print(f"{diccionario[i]}: {v:.2f}")

In [ ]:
#Aplicando todo el proceso en un único paso
list(tfidf[map(lambda x: diccionario.doc2bow(normalizar_doc_tokenize(x)), nuevo_corpus)])

### Ejercicio 3
Define una función que devuelva la matriz TF-IDF para una lista de nuevos textos (pasada como lista de *strings*) usando el diccionario y la función de normalización creadas anteriormente.

In [ ]:
def calcula_tfidf(corpus, diccionario=diccionario, normalizacion=normalizar_doc_tokenize, vectorizador=tfidf):
    """Genera la matriz TF-IDF de la lista de texto en 'corpus'
    usando el diccionario, la función de normalización y el
    vectorizador TF-IDF pasados como argumentos
    Devuelve una lista de vectores"""
    
    #COMPLETAR


In [ ]:
calcula_tfidf(nuevo_corpus)